Driving Styles

In [ ]:
import fastf1
import fastf1.plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Machine learning imports
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [12]:
fastf1.Cache.enable_cache('C:/Users/almas/OneDrive/Desktop/Purdue/Projects/F1/cache')  
year = 2025
track = 'Monza'

In [13]:
session = fastf1.get_session(year, track, 'R')
session.load()

core           INFO 	Loading data for Italian Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '16', '63', '44', '23', '5', '12', '6', '55', '87', '22', '30', '31', '10', '43', '18', '14', '27']


In [14]:
drivers = session.results['Abbreviation'].dropna().unique()
print(drivers)

['VER' 'NOR' 'PIA' 'LEC' 'RUS' 'HAM' 'ALB' 'BOR' 'ANT' 'HAD' 'SAI' 'BEA'
 'TSU' 'LAW' 'OCO' 'GAS' 'COL' 'STR' 'ALO' 'HUL']


In [ ]:
def get_lap_data(telemetry):
    if telemetry.empty:
        return None
    
    return {
        "throttle_mean": telemetry["Throttle"].mean(),
        "throttle_std": telemetry["Throttle"].std(),
        "brake_mean": telemetry["Brake"].mean(),
        "speed_mean": telemetry["Speed"].mean(),
        "speed_max": telemetry["Speed"].max(),
        "rpm_std": telemetry["RPM"].std(),
    }

In [ ]:
rows = []
telemetry_data = []

for driver in drivers:
    laps = session.laps.pick_driver(driver)
    if laps is not None and not laps.empty:
        telemetry = laps.get_telemetry()
        telemetry['Data'] = driver
        telemetry_data.append(telemetry)

all_telemetry = pd.concatenate(telemetry_data, ignore_index=True)
    

C:\Users\almas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fastf1\core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


Index(['Date', 'SessionTime', 'DriverAhead', 'DistanceToDriverAhead', 'Time',
       'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source',
       'Distance', 'RelativeDistance', 'Status', 'X', 'Y', 'Z'],
      dtype='object')


KeyError: 'LapNumber'

: 

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(driver_df)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
pca_df = pd.DataFrame(X_pca, index=driver_df.index, columns=['PC1', 'PC2'])

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)
pca_df['Cluster'] = clusters